In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras import Model, optimizers

X = tf.random.shuffle(np.random.normal(size=500).reshape(-1,10,1))
# Train_X = X[:30]
# Val_X = X[30:40]
# Test_X = X[40:50]
# Y = 2*X + 2
# Train_Y = Y[:30]
# Val_Y = Y[30:40]
# Test_Y = Y[40:50]

x_train = np.load('x_train.npy')
x_test = np.load('x_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

Train_X = x_train
Train_Y = y_train
Val_x = x_test
Val_Y = y_test

In [3]:
# inputs = tf.keras.Input(shape=(1,))
# x = Dense(2, activation="relu", name="dense_1")(inputs)
# x = Dense(2, activation="relu", name="dense_2")(x)
# outputs = Dense(1, name="predictions")(x)
# model = Model(inputs=inputs, outputs=outputs)

optimizer = optimizers.SGD(learning_rate=1e-3)
loss_fn =mean_squared_error

In [4]:
import numpy as np
from getvalues import get_values
from neural import CalphadPhaseModel
from turtle import shape
from tensorflow import keras
import tensorflow as tf
from tensorflow_addons.layers import PolynomialCrossing
from keras.layers import Conv1D
from keras.optimizers import Optimizer
import pycalphad
from pycalphad import Database, calculate, variables as v
phase_name = 'liquid'
dbf = Database('Mg_Si_Zn.tdb')
comps = ['MG', 'SI', 'ZN', 'VA']
epochs_per_run = 1
mod = pycalphad.Model(dbf, comps, phase_name)
sublattice_dof = [len(t) for t in mod.constituents]
temp_scale = x_train[:, 0].max()
energy_scale = y_train.std()
model = CalphadPhaseModel(sublattice_dof, mod.site_ratios, name=phase_name,
                                 temp_scale=temp_scale, energy_scale=energy_scale)
model.compile(optimizer=optimizer, loss='mae')

In [5]:
from sklearn.utils import shuffle
batch_size_train = 100
batch_size_val = 20

def fetch_batch(X, y, batch_size, batch):
    start = batch*batch_size
    
    X_batch = X[start:start+batch_size, :]
    y_batch = y[start:start+batch_size]
    
    return X_batch, y_batch

batch = 1

X, y = shuffle(x_train, y_train, random_state = 42)
X_batch, y_batch = fetch_batch(X, y, batch_size_train, batch)

Val_X_batch, Val_y_batch = shuffle(x_test, y_test, random_state = 42)
Val_X_batch, Val_y_batch = fetch_batch(Val_X_batch, Val_y_batch, batch_size_val, batch)



In [6]:
print(X_batch.shape, y_batch.shape)
print(Val_X_batch.shape, Val_y_batch.shape)

(100, 4) (100,)
(20, 4) (20,)


In [7]:
def get_loss(x, y, model, get_values, dbf, comps):
    with tf.GradientTape(persistent=True) as tape:
        # print(step)
        x = x[np.newaxis].T
        entropy = get_values('SM', 'LIQUID', x.T, dbf, comps)
        entropy = tf.convert_to_tensor(entropy, dtype=tf.float32)

        x_tensor = tf.convert_to_tensor(x, dtype=tf.float32)
        y_tensor = tf.convert_to_tensor(y, dtype=tf.float32)
        tape.watch(x_tensor)
        predictions = model(x_tensor, training=True)
        # print(predictions)
        dy_dx = tape.gradient(predictions, x_tensor)
        # print(dy_dx)

        RMSE_loss = tf.sqrt(tf.divide(tf.reduce_sum(tf.pow(tf.subtract(predictions, y_tensor),2.0)),tf.cast(tf.size(y_tensor), tf.float32))) 
        derivative_loss = tf.add(entropy, sum(dy_dx))
        loss_value = tf.add(RMSE_loss, derivative_loss)
    return loss_value

In [15]:
optimizer = optimizers.SGD(learning_rate=1e-3)

In [8]:
epochs = 10

for epoch in range(epochs):

    loss_array = []
    val_loss_array = []
    
    for step, (x_batch_train, y_batch_train) in enumerate(zip(X_batch, y_batch)):
        
        with tf.GradientTape(persistent=True) as tape:
            # print(step)
            x_batch_train = x_batch_train[np.newaxis].T
            entropy = get_values('SM', 'LIQUID', x_batch_train.T, dbf, comps)
            entropy = tf.convert_to_tensor(entropy, dtype=tf.float32)

            x_tensor = tf.convert_to_tensor(x_batch_train, dtype=tf.float32)
            y_tensor = tf.convert_to_tensor(y_batch_train, dtype=tf.float32)
            tape.watch(x_tensor)
            # print(x_tensor)
            predictions = model(x_tensor, training=True)
            # print(predictions)

            dy_dx = tape.gradient(predictions, x_tensor)

            RMSE_loss = tf.sqrt(tf.divide(tf.reduce_sum(tf.pow(tf.subtract(predictions, y_tensor),2.0)),tf.cast(tf.size(y_tensor), tf.float32))) 
            derivative_loss = tf.add(entropy, sum(dy_dx))
            loss_value = tf.add(RMSE_loss, derivative_loss)
            # print(loss_value)
            loss_array.append(loss_value)

            grads = tape.gradient(loss_value, model.trainable_weights)
            optimizer.apply_gradients(zip(grads, model.trainable_weights))
    #print(grads)
    # print('Epoch: ', epoch+1,' Average Train loss:', tf.get_static_value(sum(loss_array)/len(loss_array)))

    for step, (x_batch_val, y_batch_val) in enumerate(zip(Val_X_batch, Val_y_batch)):
        # val_predictions = model(x_batch_val, training=False)
        # print('Valuation predictions for the batch :')
        # print(val_predictions)
        # print('Actual Valuation for the batch :')
        # print(y_batch_val)
        # print('x_batch_val', x_batch_val)
        x, y = x_batch_val, y_batch_val
        val_loss_value = get_loss(x, y, model, get_values, dbf, comps)
        val_loss_array.append(val_loss_value)

        # y_batch_val = tf.cast(y_batch_val, tf.float32)
        # RMSE_loss = tf.sqrt(tf.divide(tf.reduce_sum(tf.pow(tf.subtract(predictions, y_tensor),2.0)),tf.cast(tf.size(y_tensor), tf.float32))) 
        # derivative_loss = tf.add(entropy, sum(dy_dx))
        # loss_value = tf.add(RMSE_loss, derivative_loss)
        # print(loss_value)
    print('Epoch: ', epoch+1, ' Average Train loss:', tf.get_static_value(sum(loss_array)/len(loss_array)), 'Average Val loss:', tf.get_static_value(sum(val_loss_array)/len(val_loss_array)))


c:\Users\zhenyap\pycalphad\pycalphad\codegen\callables.py:94: UserWarning: State variables in `build_callables` are not {N, P, T}, but {T}. This can lead to incorrectly calculated values if the state variables used to call the generated functions do not match the state variables used to create them. State variables can be added with the `additional_statevars` argument.
  warnings.warn("State variables in `build_callables` are not {{N, P, T}}, but {}. This can lead to incorrectly "


Epoch:  1  Average Train loss: [nan] Average Val loss: [nan]


KeyboardInterrupt: 

In [ ]:
print(dy_dx)

tf.Tensor(
[[3.3296045e-02]
 [3.6291695e+01]
 [3.5599747e+01]
 [3.6237251e+01]], shape=(4, 1), dtype=float32)


In [ ]:
import time
from tqdm import tqdm
from sklearn.utils import shuffle
start1 = time.time()
batch_size=100
n_epochs = 10
model1 = model
loss_history = []
acc_history = []
val_loss_history = []
val_acc_history = []
for epoch in range(n_epochs):
    X, y = shuffle(x_train, y_train, random_state = epoch**2)
    for batch in tqdm(range(len(x_train) //batch_size)):
    
        X_batch, y_batch = fetch_batch(X, y, batch_size, batch)
        # X_batch, y_batch = fetch_random_batch(X, y, batch_size)
        loss, acc = model1.train_on_batch(X_batch, y_batch)
    
    loss_history.append(loss)
    acc_history.append(acc)
    
    # Run validtion at the end of each epoch.
    y_pred = model1.predict(x_test)
    val_loss, val_acc = model1.evaluate(x_test, y_test)
    val_loss_history.append(val_loss)
    val_acc_history.append(val_acc)
        
        
    print('Epoch: %d, Train Loss %.3f, Train Acc. %.3f, Val Loss %.3f, Val Acc. %.3f' %
			(epoch+1, loss, acc, val_loss, val_acc))

ModuleNotFoundError: No module named 'tqdm'